In [1]:
import quippy

In [2]:
from quippy import descriptors

In [3]:
at = quippy.diamond(3.5, 6)
at.positions

In [4]:
at.Z

All descriptors are instantiated by a call to Descriptor(), which takes the descriptor initialisation string as its only argument. For a list of available descriptors and their parameters, see the following list, auto-generated using the following command: quip descriptor_str="--help"

In [5]:
!quip descriptor_str="--help"

In [6]:
# Pair distance descriptor between carbon atoms, with a 1.5 AA cutoff

In [7]:
desc = descriptors.Descriptor("distance_2b Z1=6 Z2=6 cutoff=4")

In [8]:
desc.n_dim # number of dimensions (1, since it's a scalar distance)

In [9]:
# Get the connectivity
at.set_cutoff(desc.cutoff()+1.0)
at.calc_connect()

In [10]:
# How many instances of the descriptor can we use in the Atoms object?
desc.count(at)

In [11]:
d = desc.calc(at)
d # the actual pairwise distances (One array is the cutoff, the other the distances)

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(d.descriptor)
plt.show()

In [13]:
n_desc, n_cross = desc.descriptor_sizes(at)
print("n_desc=%d n_cross=%d" % (n_desc,n_cross))

In [14]:
res = desc.calc(at, grad=True)
list(res)

In [15]:
res.descriptor[:, :10]

In [16]:
res.grad[:,:10]

In [17]:
res.index[:,:10]

In [18]:
# Many-body descriptor: SOAP

In [19]:
desc = descriptors.Descriptor("soap cutoff=3 l_max=4 n_max=4 atom_sigma=0.5 n_Z=1 Z={6} ")

In [20]:
at.set_cutoff(desc.cutoff())
at.calc_connect()

In [21]:
desc.descriptor_sizes(at) # Note one descriptor for each atom in the structure

In [22]:
desc.n_dim

In [23]:
desc.calc(at)

In [24]:
# Add a hydrogen atom, re-calculate the descriptor
at.add_atoms((0.2, 0.2, 0.2), (1))

In [25]:
at.calc_connect()
desc.descriptor_sizes(at) 
# Note no change in the size, as the SOAP descriptor above is just about carbons

In [26]:
# New H and C SOAP
desc = descriptors.Descriptor("soap cutoff=3 l_max=4 n_max=4 atom_sigma=0.5 n_Z=2 Z={1 6} n_species=2 species_Z={1 6}")

In [27]:
desc.descriptor_sizes(at) # Note the size has gone up

In [28]:
desc.calc(at)

#### We can get the same descriptor using the QUIP program, in which the input is an extended xyz file and the specification as before

In [29]:
at.write("diamondH.xyz")

In [30]:
!quip atoms_filename=diamondH.xyz descriptor_str="soap cutoff=3 l_max=4 n_max=4 atom_sigma=0.5 n_Z=2 Z={1 6} n_species=2 species_Z={1 6}"

In [31]:
# ^ Each vector is on its own line, starting with a DESC

In [32]:
# Three-site benzene monomer
desc_monomer = descriptors.Descriptor('general_monomer signature={6 6 6} atom_ordercheck=F')

In [33]:
desc_monomer.n_dim # Three distances -> dimensionality 3

In [34]:
benzat = quippy.AtomsList('benzene_frames.xyz') # some benzenes

In [35]:
from ase.visualize import view
from ase.atoms import Atoms as AseAtoms 

firstBenzene = benzat[0]
view(AseAtoms(firstBenzene), viewer='x3d')

In [36]:
 # Doesn't look much like benzene to me, but what do I know?

In [37]:
benzat.pos.shape

In [38]:
for i in range(len(benzat)):
    benzat[i].set_cutoff(desc_monomer.cutoff()+0.5)
    benzat[i].calc_connect()

In [39]:
res = desc_monomer.calc(benzat, grad=True)[0]

In [40]:
res.grad.shape